In [5]:
import cv2
import numpy as np
from load_deepdeform import load_deepdeform_data

images, depth_maps, poses, times, render_poses, render_times, hwff, i_split = load_deepdeform_data('./data/human', True, 1, 'spiral')
#print(f"[Info] Loaded DeepDeform:\n\t\timages.shape: {images.shape}\n\t\trender_poses.shape: {render_poses.shape}\n\t\thwff: {hwff}\n\t\targs.datadir: {args.datadir}")
i_train, i_val, i_test = i_split
near = 0.1
far = np.max(depth_maps) + 0.1
print("===HERE===")
print(f"[Info] Setting near plane at distance {near} and far plane at distance {far}.")
print(depth_maps)

===HERE===
[Info] Setting near plane at distance 0.1 and far plane at distance 3608.85.
[[[[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [   0.  ]
   [   0.  ]
   [   0.  ]]

  [[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [   0.  ]
   [   0.  ]
   [   0.  ]]

  [[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [   0.  ]
   [   0.  ]
   [   0.  ]]

  ...

  [[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [1509.  ]
   [1505.75]
   [   0.  ]]

  [[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [1509.75]
   [1509.5 ]
   [   0.  ]]

  [[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [   0.  ]
   [ 754.  ]
   [   0.  ]]]


 [[[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [   0.  ]
   [   0.  ]
   [   0.  ]]

  [[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [   0.  ]
   [   0.  ]
   [   0.  ]]

  [[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [   0.  ]
   [   0.  ]
   [   0.  ]]

  ...

  [[   0.  ]
   [   0.  ]
   [   0.  ]
   ...
   [1513.5 ]
   [1510.75]
   [   0.  ]]

  [[   0.  ]
   [   0.  ]
